# Stock prediction: 1D convolution

In [1]:
import pandas as pd
import numpy as np
import pandas_market_calendars as mcal
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import InputLayer

from tensorflow.keras.utils import plot_model

from utils import *

# for Robust location estimation
from statsmodels import robust
import statsmodels.api as sm
from astropy.stats import biweight_location

tf.random.set_seed(25)

In [2]:
prices = pd.read_csv("data/SX5E/close_mat.csv")
composition = pd.read_csv("data/SX5E/SX5E Index_compositionMat.csv")
capitalization = pd.read_csv("data/SX5E/capitalization_mat.csv")

prices = setup_tables(prices)
composition = setup_tables(composition)
capitalization = setup_tables(capitalization)

prices = select_time_slice(prices, start=20140101, end=20181231)
capitalization = select_time_slice(capitalization, start=20140101, end=20181231)
composition = select_time_slice(composition, start=20140101, end=20181231)

prices.set_index("Date", drop=True, inplace=True)
composition.set_index("Date", drop=True, inplace=True)
capitalization.set_index("Date", drop=True, inplace=True)

In [3]:
stocks = get_full_time_stock_in_period(composition.iloc[:, 1:])
print("Number of stocks =", len(stocks))

Number of stocks = 40


In [4]:
trading_days = get_trading_dates(start_period="2014-01-01", end_period="2018-12-31", market="EUREX")
print("Total number of trading days =", len(trading_days))

Total number of trading days = 1272


In [5]:
prices = prices[stocks].loc[trading_days]
capitalization = capitalization[stocks].loc[trading_days]
composition = composition[stocks].loc[trading_days]

In [6]:
price_to_returns(prices)

,VOW3 GY Equity,ENGI FP Equity,BAYN GY Equity,BNP FP Equity,PHIA NA Equity,ISP IM Equity,SAN FP Equity,MC FP Equity,ENEL IM Equity,CS FP Equity,...,GLE FP Equity,ITX SQ Equity,OR FP Equity,BBVA SQ Equity,ASML NA Equity,SAP GY Equity,INGA NA Equity,ENI IM Equity,ABI BB Equity,DTE GY Equity
2014-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-03,0.000000,0.001795,0.007496,0.002521,0.004668,0.014582,0.013826,-0.001905,0.014040,0.003804,...,-0.004697,0.002530,0.007564,-0.003199,-0.000148,0.000649,0.000496,0.009238,0.011242,0.011542
2014-01-06,-0.008728,-0.011944,-0.004464,0.001616,0.007805,0.010503,0.001311,-0.018702,0.002517,-0.002021,...,0.009316,-0.005046,-0.009087,0.010659,-0.025092,0.004699,-0.010312,0.005721,-0.017918,-0.006520
2014-01-07,0.006289,0.012390,0.010463,0.028689,0.007929,0.032823,-0.001179,-0.007001,0.012555,0.025316,...,0.040278,0.009721,0.005582,0.057042,-0.008024,-0.007903,0.035467,0.004551,0.020109,0.004922
2014-01-08,0.000750,0.005970,-0.012820,0.014293,0.001829,0.021716,-0.012325,-0.014493,0.004960,0.006914,...,0.029730,-0.006697,-0.018636,0.006973,0.002747,0.016908,0.021287,-0.016421,0.004830,-0.002041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-21,-0.004169,0.003610,0.006393,0.002538,-0.002107,-0.016864,-0.006172,0.013395,0.007382,0.005779,...,0.005729,0.003972,-0.004464,-0.007628,-0.013355,-0.015381,0.014214,-0.003761,-0.001508,-0.007603
2018-12-24,0.000000,-0.018385,0.000000,-0.017975,-0.013478,0.000000,-0.019027,-0.032748,0.000000,-0.001490,...,-0.017088,-0.021978,-0.015197,-0.013100,-0.015792,0.000000,-0.018019,0.000000,-0.018798,0.000000
2018-12-27,-0.040893,-0.021173,-0.036325,-0.006316,-0.002304,-0.001673,-0.006465,0.008158,-0.026610,-0.012148,...,-0.014306,-0.002697,-0.010372,-0.002852,0.013295,-0.007290,-0.013735,-0.018441,-0.018987,-0.027981
2018-12-28,0.010768,0.021215,0.023665,0.021533,0.012869,0.016134,0.019794,0.022658,-0.000792,0.012945,...,0.021312,0.009013,0.012781,0.021122,0.030161,0.013288,0.018170,0.016864,0.002616,0.015764


## Example with single stock

In [9]:
price = prices.iloc[:, 0]
returns = price_to_returns(price, log=True)

In [10]:
returns

2014-01-02         NaN
2014-01-03    0.000000
2014-01-06   -0.008766
2014-01-07    0.006270
2014-01-08    0.000750
                ...   
2018-12-21   -0.004178
2018-12-24    0.000000
2018-12-27   -0.041753
2018-12-28    0.010711
2018-12-31    0.000000
Name: VOW3 GY Equity, Length: 1272, dtype: float64